NYCTaxi
------

Presentado por: Yosef Guevara Salamanca

### Ejercicio

Taxi trips taken in 2013 released by a FOIA request.  Around 20GB CSV uncompressed.

[Download link](http://www.andresmh.com/nyctaxitrips/)

**Try the following:**

*  Use `dask.dataframe` with pandas-style queries
*  Store in HDF5 both with and without categoricals, measure the size of the file and query times
*  Set the index by one of the date-time columns and store in castra (also using categoricals).  Perform range queries and measure speed.  What size and complexity of query can you perform while still having an "interactive" experience?

### Estableciendo cliente dask

Lo primero que se realizara sera generar un cliente que contará con 4 workers y 4 hilos por worker con un máximo de memoria a utilizar de 1GB

In [1]:
from dask.distributed import Client
client = Client(n_workers=2, threads_per_worker=8, memory_limit='3GB')
client

Connection method: Cluster object,Cluster type: LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Status: running,Using processes: True
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 16,Total memory: 5.59 GiB
Comm: tcp://127.0.0.1:53111,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 5.59 GiB
Comm: tcp://127.0.0.1:53123,Total threads: 8
Dashboard: http://127.0.0.1:53124/status,Memory: 2.79 GiB
Nanny: tcp://127.0.0.1:53113,


In [2]:
## Librerias necesarias

from glob import glob
from dask import delayed
from dask import dataframe as dd

### Lectura de los datos

In [3]:
## Se visulizan los primeros 5 filas del dataframe

nyc = dd.read_csv('D:\\Descargas\\Taxi\\*.csv')
nyc.head()

,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,89D227B655E5C82AECF13C3F540D4CF4,BA96DE419E711691B9445D6A6307C170,CMT,1,N,2013-01-01 15:11:48,2013-01-01 15:18:10,4,382,1.0,-73.978165,40.757977,-73.989838,40.751171
1,0BD7C8F5BA12B88E0B67BED28BEA73D8,9FD8F69F0804BDB5549F40E9DA1BE472,CMT,1,N,2013-01-06 00:18:35,2013-01-06 00:22:54,1,259,1.5,-74.006683,40.731781,-73.994499,40.750660
2,0BD7C8F5BA12B88E0B67BED28BEA73D8,9FD8F69F0804BDB5549F40E9DA1BE472,CMT,1,N,2013-01-05 18:49:41,2013-01-05 18:54:23,1,282,1.1,-74.004707,40.737770,-74.009834,40.726002
3,DFD2202EE08F7A8DC9A57B02ACB81FE2,51EE87E3205C985EF8431D850C786310,CMT,1,N,2013-01-07 23:54:15,2013-01-07 23:58:20,2,244,0.7,-73.974602,40.759945,-73.984734,40.759388
4,DFD2202EE08F7A8DC9A57B02ACB81FE2,51EE87E3205C985EF8431D850C786310,CMT,1,N,2013-01-07 23:25:03,2013-01-07 23:34:24,1,560,2.1,-73.976250,40.748528,-74.002586,40.747868


### 1. Use `dask.dataframe` with pandas-style queries

In [4]:
## Trips mayores a los 10 km

nyc[nyc.trip_distance > 10].head()

,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
7,0B57B9633A2FECD3D3B1944AFC7471CF,CCD4367B417ED6634D986F573A552A62,CMT,1,N,2013-01-07 12:39:18,2013-01-07 13:10:56,3,1898,10.7,-73.989937,40.756775,-73.865250,40.770630
20,D8AADA4E722C87C10E609654612630DD,8F1E4F779A4D67431C3FCFA4E70A87DC,CMT,2,N,2013-01-10 23:41:51,2013-01-11 00:09:11,1,1640,17.5,-73.783310,40.648766,-73.988914,40.748207
28,8E189DABE265CC03FEF4BFC695B32559,906939B31C0B93768242A58A682412B8,CMT,1,N,2013-01-01 14:02:01,2013-01-01 14:27:39,2,1537,10.2,-73.862709,40.769142,-73.982079,40.762295
38,CD9DEF073BAB75B8B36015D85FD3F777,80430F0667E3C82E63AAF4F0DC547664,CMT,1,N,2013-01-05 03:20:28,2013-01-05 03:43:36,4,1387,11.2,-73.991142,40.750366,-73.984436,40.618481
46,8F9B593AE29F10B40EBEA0BBDEDD249F,61AF8331CB51E436B747478FC0E78B67,CMT,1,N,2013-01-07 17:38:07,2013-01-07 17:57:46,1,1179,10.3,-73.873154,40.773998,-73.928780,40.859314


In [5]:
## Viajes con un numero de pasajeros igual a 3.

nyc[nyc.passenger_count == 3].head()

,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
7,0B57B9633A2FECD3D3B1944AFC7471CF,CCD4367B417ED6634D986F573A552A62,CMT,1,N,2013-01-07 12:39:18,2013-01-07 13:10:56,3,1898,10.7,-73.989937,40.756775,-73.865250,40.770630
37,764CA5AE502C0FEC9BB69DBCB6BB4ECC,26516CD746248B3495CBD99A8ABAF8B4,CMT,1,N,2013-01-05 20:51:53,2013-01-05 21:02:43,3,650,3.3,-73.993828,40.721008,-73.952400,40.723228
66,0CC08DE5645A7137059499EA11886805,4F2627F9FD34A9B9EFCCFC5147048345,CMT,1,N,2013-01-07 18:45:14,2013-01-07 18:52:54,3,459,0.9,-73.979706,40.758427,-73.985558,40.747265
77,4C005EEBAA7BF26B84B21586332488A2,351BE7D984BE17DB2FA80A748E816472,CMT,1,N,2013-01-07 21:58:43,2013-01-07 22:08:42,3,598,2.0,-73.987099,40.755215,-73.974457,40.755589
78,5C1E310A26EEE17AA72AFE63194DE5B5,A56B275487B0D09E6BA4C8801E0B2BE3,CMT,1,N,2013-01-01 17:11:51,2013-01-01 17:17:18,3,327,1.4,-73.987930,40.749542,-73.978981,40.766735


### 2. Store in HDF5 both with and without categoricals, measure the size of the file and query times

In [6]:
## Se desactiva el cliente para permitir la escritura del archivo hdf5

client.shutdown()

In [7]:
## Se importa la libreria h5py

import h5py

In [8]:
## Se almacenan los datos como hdf5

hdf = h5py.File("D:\\Descargas\\Taxi\\Taxi.hdf5","a")
hdf

<HDF5 file "Taxi.hdf5" (mode r+)>

In [9]:
## Se reactiva el cliente

from dask.distributed import Client
client = Client(n_workers=2, threads_per_worker=8, memory_limit='3GB')
client

Connection method: Cluster object,Cluster type: LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Status: running,Using processes: True
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 16,Total memory: 5.59 GiB
Comm: tcp://127.0.0.1:53146,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 5.59 GiB
Comm: tcp://127.0.0.1:53161,Total threads: 8
Dashboard: http://127.0.0.1:53163/status,Memory: 2.79 GiB
Nanny: tcp://127.0.0.1:53148,


In [10]:
## Se raliza la lectura del archivo hdf5 creado

data_hdf = dd.read_hdf("D:\\Descargas\\Taxi","\\Taxi.hdf5")

ImportError: Missing optional dependency 'tables'.  Use pip or conda to install tables.

Nota: No fue posible leer el archivo creado menciona una libreria llamada **tables** que se encuentra obsoleta, se intenta con la nueva libreria **pytables**, pero nunca conecta al servidor por lo que fue imposible cargar el archivo como se muestra en la siguiente imagen.

![tables](./img/tables.png) 

Se procede entonces a trabajar con el dataframe ya ejecutado

In [11]:
## Se verifica el tipo de variables del DataSet

nyc.dtypes

medallion              object
hack_license           object
vendor_id              object
rate_code               int64
store_and_fwd_flag     object
pickup_datetime        object
dropoff_datetime       object
passenger_count         int64
trip_time_in_secs       int64
trip_distance         float64
pickup_longitude      float64
pickup_latitude       float64
dropoff_longitude     float64
dropoff_latitude      float64
dtype: object

In [13]:
## Se crea una lista que contiene las varaibles no categoricas

non_categorical =['rate_code',
                 'passenger_count',
                 'trip_time_in_secs',
                 'trip_distance',
                 'pickup_longitude',
                 'pickup_latitude',
                 'dropoff_longitude',
                 'dropoff_latitude']


Se miden los tiempos del querry del dataset con los datos categoricos

In [14]:
%%time
nyc
nyc.head()

Wall time: 593 ms


,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,89D227B655E5C82AECF13C3F540D4CF4,BA96DE419E711691B9445D6A6307C170,CMT,1,N,2013-01-01 15:11:48,2013-01-01 15:18:10,4,382,1.0,-73.978165,40.757977,-73.989838,40.751171
1,0BD7C8F5BA12B88E0B67BED28BEA73D8,9FD8F69F0804BDB5549F40E9DA1BE472,CMT,1,N,2013-01-06 00:18:35,2013-01-06 00:22:54,1,259,1.5,-74.006683,40.731781,-73.994499,40.750660
2,0BD7C8F5BA12B88E0B67BED28BEA73D8,9FD8F69F0804BDB5549F40E9DA1BE472,CMT,1,N,2013-01-05 18:49:41,2013-01-05 18:54:23,1,282,1.1,-74.004707,40.737770,-74.009834,40.726002
3,DFD2202EE08F7A8DC9A57B02ACB81FE2,51EE87E3205C985EF8431D850C786310,CMT,1,N,2013-01-07 23:54:15,2013-01-07 23:58:20,2,244,0.7,-73.974602,40.759945,-73.984734,40.759388
4,DFD2202EE08F7A8DC9A57B02ACB81FE2,51EE87E3205C985EF8431D850C786310,CMT,1,N,2013-01-07 23:25:03,2013-01-07 23:34:24,1,560,2.1,-73.976250,40.748528,-74.002586,40.747868


Se miden los tiempos del querry del dataset sin los datos categoricos

In [15]:
%%time
nyc_non = nyc[non_categorical]
nyc_non.head()

Wall time: 461 ms


,rate_code,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,1,4,382,1.0,-73.978165,40.757977,-73.989838,40.751171
1,1,1,259,1.5,-74.006683,40.731781,-73.994499,40.750660
2,1,1,282,1.1,-74.004707,40.737770,-74.009834,40.726002
3,1,2,244,0.7,-73.974602,40.759945,-73.984734,40.759388
4,1,1,560,2.1,-73.976250,40.748528,-74.002586,40.747868


In [16]:
## Se crea un segundo archivo con los datos no categoricos

h5py.File("D:\\Descargas\\Taxi\\Taxi_non_cat.hdf5","a")

<HDF5 file "Taxi_non_cat.hdf5" (mode r+)>

### 3. Set the index by one of the date-time columns and store in castra (also using categoricals). Perform range queries and measure speed. What size and complexity of query can you perform while still having an "interactive" experience?

Este dataset no posee columnas de fecha o tiempo como se puede observa más arriba, por lo que simplemente se procede a hacer la compración de los tiempos con algunas querys timpo pandas donde se aprecia una mayor velocidad en el dataframe qu no posee datos categoricos

In [17]:
%%time

## Trips menores a los 5 km para el dataset sin datos categoricos

nyc_non[nyc_non.trip_distance < 5].head()

Wall time: 467 ms


,rate_code,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,1,4,382,1.0,-73.978165,40.757977,-73.989838,40.751171
1,1,1,259,1.5,-74.006683,40.731781,-73.994499,40.750660
2,1,1,282,1.1,-74.004707,40.737770,-74.009834,40.726002
3,1,2,244,0.7,-73.974602,40.759945,-73.984734,40.759388
4,1,1,560,2.1,-73.976250,40.748528,-74.002586,40.747868


In [18]:
%%time

## Trips menores a los 5 km para el dataset con datos categoricos

nyc[nyc.trip_distance < 5].head()

Wall time: 626 ms


,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,89D227B655E5C82AECF13C3F540D4CF4,BA96DE419E711691B9445D6A6307C170,CMT,1,N,2013-01-01 15:11:48,2013-01-01 15:18:10,4,382,1.0,-73.978165,40.757977,-73.989838,40.751171
1,0BD7C8F5BA12B88E0B67BED28BEA73D8,9FD8F69F0804BDB5549F40E9DA1BE472,CMT,1,N,2013-01-06 00:18:35,2013-01-06 00:22:54,1,259,1.5,-74.006683,40.731781,-73.994499,40.750660
2,0BD7C8F5BA12B88E0B67BED28BEA73D8,9FD8F69F0804BDB5549F40E9DA1BE472,CMT,1,N,2013-01-05 18:49:41,2013-01-05 18:54:23,1,282,1.1,-74.004707,40.737770,-74.009834,40.726002
3,DFD2202EE08F7A8DC9A57B02ACB81FE2,51EE87E3205C985EF8431D850C786310,CMT,1,N,2013-01-07 23:54:15,2013-01-07 23:58:20,2,244,0.7,-73.974602,40.759945,-73.984734,40.759388
4,DFD2202EE08F7A8DC9A57B02ACB81FE2,51EE87E3205C985EF8431D850C786310,CMT,1,N,2013-01-07 23:25:03,2013-01-07 23:34:24,1,560,2.1,-73.976250,40.748528,-74.002586,40.747868
